# 5. Analysis of e-waste generation in the EU

This notebook contains the analysis of the data regarding the e-weaste generated in the European Union from 1980 to 2021. A new data frame is created from the clean data. 

Relevant variables include:

1. E-waste description
2. E-waste generation in the EU
3. GDP per country
4. GDP per country (per capita)
5. E-waste categories


In [1]:
import pandas as pd
import numpy as np
import pymysql
import getpass
import matplotlib
import matplotlib.pyplot as plt
import mysql_connection as mc
%matplotlib inline

In [2]:
description = pd.read_csv("../data/clean_data/1_eee_description.csv")

In [4]:
ewaste = pd.read_csv("../data/clean_data/2_ewaste_generation.csv")

In [6]:
gdp_t = pd.read_csv("../data/clean_data/4_gdp_total_countries.csv")

In [7]:
gdp_pc = pd.read_csv("../data/clean_data/5_gdp_per_capita.csv")

In [5]:
countries = pd.read_csv("../data/clean_data/6_countries.csv")

In [3]:
cat = pd.read_csv("../data/clean_data/7_ewaste_category.csv")

In [8]:
df = description.merge(right=cat, 
                       how="left", 
                       on="category")[["eee_key", 
                                       "eee_description", 
                                       "type"]]

In [9]:
df2 = ewaste.merge(right=df, 
                   how="left", 
                   on="eee_key")[["year",
                                  "country",
                                  "inhabitants",
                                  "ewaste_t",
                                  "ewaste_pieces",
                                  "ewaste_kpi",
                                  "ewaste_ppi",
                                  "eee_key",
                                  "eee_description",
                                  "type"]]

In [10]:
df3 = df2.merge(right=countries, 
                  how="left", 
                  on="country")[["country_name",
                                 "year",
                                 "income_group", 
                                 "inhabitants",
                                 "ewaste_t", 
                                 "ewaste_pieces", 
                                 "ewaste_kpi", 
                                 "ewaste_ppi", 
                                 "eee_key", 
                                 "eee_description",
                                 "type"]]

In [11]:
df4 = df3.merge(right=gdp_t, 
                  how="left", 
                  on=["country_name", "year"])[["country_name",
                                                "year",
                                                "income_group",
                                                "inhabitants",
                                                "ewaste_t",
                                                "ewaste_pieces",
                                                "ewaste_kpi",
                                                "ewaste_ppi",
                                                "gdp",
                                                "eee_key",
                                                "eee_description",
                                                "type"]]

In [12]:
df5 = df4.merge(right=gdp_pc, 
                  how="left", 
                  on=["country_name", "year"])[["country_name",
                                                "year",
                                                "income_group",
                                                "inhabitants",
                                                "ewaste_t",
                                                "ewaste_pieces",
                                                "ewaste_kpi",
                                                "ewaste_ppi",
                                                "gdp", 
                                                "gdp_pc",
                                                "eee_key",
                                                "eee_description",
                                                "type"]]

In [13]:
df5.head()

,country_name,year,income_group,inhabitants,ewaste_t,ewaste_pieces,ewaste_kpi,ewaste_ppi,gdp,gdp_pc,eee_key,eee_description,type
0,Austria,1980,High income,7540000.0,13.658216,443,0.001811,0.000059,8.205891e+10,10869.546420,1,Central Heating (household installed),Large equipment
1,Austria,1981,High income,7556000.0,54.909780,1780,0.007267,0.000236,7.103423e+10,9385.249018,1,Central Heating (household installed),Large equipment
2,Austria,1982,High income,7565000.0,123.893263,4017,0.016377,0.000531,7.127529e+10,9410.347225,1,Central Heating (household installed),Large equipment
3,Austria,1983,High income,7543000.0,219.884724,7127,0.029151,0.000945,7.212102e+10,9537.407431,1,Central Heating (household installed),Large equipment
4,Austria,1984,High income,7544000.0,341.413262,11067,0.045256,0.001467,6.798535e+10,8991.065076,1,Central Heating (household installed),Large equipment
...,...,...,...,...,...,...,...,...,...,...,...,...,...
74083,Sweden,2024,High income,NaN,371.385167,4025,NaN,NaN,NaN,NaN,1002,"Cooled Dispensers (e.g. for vending, cold drinks)",Temperature exchange equipment
74084,Sweden,2025,High income,NaN,341.513497,3702,NaN,NaN,NaN,NaN,1002,"Cooled Dispensers (e.g. for vending, cold drinks)",Temperature exchange equipment
74085,Sweden,2026,High income,NaN,311.116664,3373,NaN,NaN,NaN,NaN,1002,"Cooled Dispensers (e.g. for vending, cold drinks)",Temperature exchange equipment
74086,Sweden,2027,High income,NaN,280.852257,3043,NaN,NaN,NaN,NaN,1002,"Cooled Dispensers (e.g. for vending, cold drinks)",Temperature exchange equipment


In [14]:
# Create a column of e-waste generated in million tonnes 

df5["ewaste_mt"] = df5["ewaste_t"] / 1000000

# Create a column with e-waste equivalent to Eiffel Towers

eiffel_tower = 10100
df5["ewaste_eff"] = df5["ewaste_mt"] * 1000000 / eiffel_tower

In [15]:
# Let's look at the waste generated per year in the whole region

year_mt = (df5
           .groupby("year")
           .agg({"ewaste_mt": "sum", "ewaste_eff": "sum"})
           .reset_index())

In [16]:
year_mt

,year,ewaste_mt,ewaste_eff
0,1980,0.059565,5.897494
1,1981,0.159199,15.762256
2,1982,0.292531,28.963419
3,1983,0.456692,45.217031
4,1984,0.650159,64.372141
5,1985,0.870613,86.199340
6,1986,1.115064,110.402424
7,1987,1.380064,136.640037
8,1988,1.662062,164.560641
9,1989,1.957516,193.813481


In [17]:
# Let's look at the e-waste equivalent to Eiffel Towers generated in a single year

def e_waste_equiv_et(year):
    print("In " + str(year) + ", " + str(year_mt.loc[year_mt["year"] == year]["ewaste_mt"].iloc[0]) 
          + " million metric tonnes of e-waste was generated. This is equivalent to " 
          + str(year_mt.loc[year_mt["year"] == year]["ewaste_eff"].iloc[0]) + " Eiffel Towers!")

In [18]:
e_waste_equiv_et(2000)
e_waste_equiv_et(2020)

In 2000, 5.502341995322558 million metric tonnes of e-waste was generated. This is equivalent to 544.7863361705496 Eiffel Towers!
In 2020, 10.517656224831697 million metric tonnes of e-waste was generated. This is equivalent to 1041.3521014684857 Eiffel Towers!


In [29]:
# Let's look at the numbers per category in a given year

ewaste_cat = (df5.loc[df5["year"] == 2019]
              .groupby("eee_key")
              .agg({"ewaste_mt": "sum", "ewaste_eff": "sum", "ewaste_pieces": "sum"})
              .reset_index()).merge(right=df, 
                                    how="left", 
                                    on="eee_key")[["eee_key", 
                                                   "eee_description", 
                                                   "ewaste_mt", 
                                                   "ewaste_eff", 
                                                   "ewaste_pieces"]]

In [30]:
ewaste_cat.sort_values(by=["ewaste_mt"])

,eee_key,eee_description,ewaste_mt,ewaste_eff,ewaste_pieces
48,801,"Household Medical equipment(e.g. thermometers,...",0.000728,0.072102,4045746
33,406,"Cameras (e.g. camcorders, photo & digital stil...",0.007206,0.713515,20877171
47,703,"Leisure equipment (e.g. sports equipment, elec...",0.007532,0.745779,1020688
39,504,"Special Lamps (e.g. professional mercury, high...",0.011315,1.120289,144924940
52,1001,"Non- cooled Dispensers (e.g. for vending, hot ...",0.011693,1.157770,265759
40,505,LED Lamps (incl. retrofit LED lamps),0.012146,1.202589,151826878
29,402,"Portable Audio & Video (e.g. MP3, e-readers, c...",0.015041,1.489159,56152760
24,306,"Mobile Phones (incl. smartphones, pagers)",0.016556,1.639248,183519160
37,502,Compact Fluorescent Lamps (incl. retrofit & no...,0.025189,2.494002,313450006
2,101,Professional Heating & Ventilation (excl. cool...,0.025546,2.529295,291957


In [26]:
# Let's look at the numbers in 2019

ewaste_cat_2019 = (df5.loc[df5["year"] == 2019]
              .groupby("eee_key")
              .agg({"ewaste_mt": "sum", "ewaste_eff": "sum", "ewaste_pieces": "sum"})
              .reset_index()).merge(right=df, 
                                    how="left", 
                                    on="eee_key")[["eee_key", 
                                                   "eee_description", 
                                                   "ewaste_mt", 
                                                   "ewaste_eff", 
                                                   "ewaste_pieces"]]

In [28]:
ewaste_cat_2019.sort_values(by=["ewaste_mt"])

,eee_key,eee_description,ewaste_mt,ewaste_eff,ewaste_pieces
48,801,"Household Medical equipment(e.g. thermometers,...",0.000728,0.072102,4045746
33,406,"Cameras (e.g. camcorders, photo & digital stil...",0.007206,0.713515,20877171
47,703,"Leisure equipment (e.g. sports equipment, elec...",0.007532,0.745779,1020688
39,504,"Special Lamps (e.g. professional mercury, high...",0.011315,1.120289,144924940
52,1001,"Non- cooled Dispensers (e.g. for vending, hot ...",0.011693,1.157770,265759
40,505,LED Lamps (incl. retrofit LED lamps),0.012146,1.202589,151826878
29,402,"Portable Audio & Video (e.g. MP3, e-readers, c...",0.015041,1.489159,56152760
24,306,"Mobile Phones (incl. smartphones, pagers)",0.016556,1.639248,183519160
37,502,Compact Fluorescent Lamps (incl. retrofit & no...,0.025189,2.494002,313450006
2,101,Professional Heating & Ventilation (excl. cool...,0.025546,2.529295,291957


In [ ]:
# Let's look at the numbers in Germany in 2019

(df5.loc[df5["country_name"] == "Germany"]).loc[df5["year"] == 2019][["eee_key", 
                           "eee_description", 
                           "ewaste_mt", 
                           "ewaste_eff", 
                           "ewaste_pieces"]].sort_values(by=["ewaste_mt"], ascending=False)

In [ ]:
ewaste_cat_1

In [ ]:
# Let's look at the numbers per type (combined categories) in a given year

ewaste_type = (df5
               .loc[df5["year"] == 2019]
               .groupby("type")
               .agg({"ewaste_mt": "sum", "ewaste_eff": "sum"})
               .reset_index())

In [ ]:
ewaste_type.sort_values(by=["ewaste_mt"])

In [ ]:
# Let's look at the numbers per country in a given year

ewaste_countries = (df5
                    .groupby(["country_name", "year"])
                    .agg({"ewaste_mt": "sum", "ewaste_eff": "sum"})
                    .reset_index())

In [ ]:
ewaste_countries

In [ ]:
# Let's look at the numbers per country in a given year

(ewaste_countries[ewaste_countries["country_name"] == "Germany"])

In [ ]:
# df5.to_csv("../Data/clean_data/10_ewaste_eu.csv", index=False)